In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
import MeCab
import ipadic

In [2]:
df = pd.read_csv("./errors.csv")
df.head()

,Item.Description,Item.Manufacturer,Item.PartsNo,問題点.No,問題点.内容,問題点.原因,問題点.発覚日時,問題点.Project,対策.応急処置,対策.恒久対策
0,サンプリングライン,C社,P-891516,P001,配管の亀裂,定期点検の不備,2025/07/12,塩素注入設備更新プロジェクト,フィルターの交換,材質を耐塩素性の高いものに変更
1,ポンプ,F社,P-689038,P002,腐食による機能不全,異物の混入,2025/07/25,塩素注入設備更新プロジェクト,漏れ箇所の部品交換,定期点検マニュアルの更新
2,フィルター,D社,P-155322,P003,配管の亀裂,異物の混入,2025/07/09,塩素注入設備更新プロジェクト,センサーの再校正,材質を耐塩素性の高いものに変更
3,電磁弁,A社,P-426572,P004,制御盤が停止,締め付けトルクの不足,2025/07/25,塩素注入設備更新プロジェクト,詰まり箇所の洗浄,自動洗浄機能の導入を検討
4,圧力計,E社,P-889925,P005,制御盤が停止,締め付けトルクの不足,2025/07/30,塩素注入設備更新プロジェクト,センサーの再校正,防食・防湿処理を強化


In [3]:
df_X = df.drop(
    [
        "問題点.原因",
        "問題点.No",
        "対策.応急処置",
        "対策.恒久対策",
    ],
    axis=1,
)
df_y = df["問題点.原因"]

text_columns = [
    "Item.Description", 
    "問題点.内容",
]
categorical_columns = ["Item.Manufacturer", "Item.PartsNo", "問題点.Project"]

df_X.head()

,Item.Description,Item.Manufacturer,Item.PartsNo,問題点.内容,問題点.発覚日時,問題点.Project
0,サンプリングライン,C社,P-891516,配管の亀裂,2025/07/12,塩素注入設備更新プロジェクト
1,ポンプ,F社,P-689038,腐食による機能不全,2025/07/25,塩素注入設備更新プロジェクト
2,フィルター,D社,P-155322,配管の亀裂,2025/07/09,塩素注入設備更新プロジェクト
3,電磁弁,A社,P-426572,制御盤が停止,2025/07/25,塩素注入設備更新プロジェクト
4,圧力計,E社,P-889925,制御盤が停止,2025/07/30,塩素注入設備更新プロジェクト


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.2, random_state=42
)

print(f"学習データ数: {len(X_train)}")
print(f"テストデータ数: {len(X_test)}")

学習データ数: 160
テストデータ数: 40


In [5]:
def tokenize_text(text):
    tagger = MeCab.Tagger(f"-O wakati {ipadic.MECAB_ARGS}")
    return tagger.parse(text).strip()

In [6]:
tfidf_vectorizers = {}
X_tfidf_features = []

for col in text_columns:
    print(f"processing text column: {col}")
    cleaned_text = df_X[col].fillna("").apply(tokenize_text)
    vectorizer = TfidfVectorizer(min_df=1, max_df=0.9, ngram_range=(1, 2))
    X_tfidf_features.append(vectorizer.fit_transform(cleaned_text))
    tfidf_vectorizers[col] = vectorizer

processing text column: Item.Description
processing text column: 問題点.内容


In [7]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
X_categorical_features = encoder.fit_transform(df_X[categorical_columns])

print(f"カテゴリ特徴量の形状: {X_categorical_features.shape}")

カテゴリ特徴量の形状: (200, 207)


In [8]:
all_features = hstack(X_tfidf_features + [X_categorical_features])

print(f"結合された全特徴量の形状: {all_features.shape}")

結合された全特徴量の形状: (200, 263)


In [9]:
X_train_combined, X_test_combined, y_train, y_test = train_test_split(
    all_features,
    df_y,
    test_size=0.2,
    random_state=42,
)

print(f"\n最終的な学習データ数: {X_train_combined.shape[0]}")
print(f"最終的なテストデータ数: {X_test_combined.shape[0]}")


最終的な学習データ数: 160
最終的なテストデータ数: 40


In [10]:
model = SVC(kernel="linear", C=1.0, random_state=42)
model.fit(X_train_combined, y_train)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [11]:
y_pred = model.predict(X_test_combined)

print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

   パッキンの経年劣化       0.00      0.00      0.00         3
     塩素ガスの侵入       0.29      0.33      0.31         6
   外的要因による損傷       0.14      0.25      0.18         4
     定期点検の不備       0.00      0.00      0.00         7
     水質変動の影響       0.25      0.20      0.22         5
       異物の混入       0.00      0.00      0.00         1
  締め付けトルクの不足       0.33      0.50      0.40         4
     部品の初期不良       0.00      0.00      0.00         8
    電源系統の不安定       0.00      0.00      0.00         2

    accuracy                           0.15        40
   macro avg       0.11      0.14      0.12        40
weighted avg       0.12      0.15      0.13        40



In [12]:
y_pred_dec = model.decision_function(X_test_combined)
X_test_labels = X_test.reset_index().drop(["index"], axis=1)
class_labels = model.classes_
for i in range(len(y_pred_dec)):
    y_sorted = sorted(list(zip(class_labels, y_pred_dec[i])), key=lambda x: x[1], reverse=True)
    y_sorted = list(map(lambda x: (x[0], round(float(x[1]), 3)), y_sorted))
    print(X_test_labels.iloc[i])
    print(y_sorted[:3])
    print(y_test.iloc[i])
    print("------------------------------------------------------")

Item.Description                流量計
Item.Manufacturer                B社
Item.PartsNo               P-775529
問題点.内容                       制御盤が停止
問題点.発覚日時                 2025/07/02
問題点.Project          塩素注入設備更新プロジェクト
Name: 0, dtype: object
[('締め付けトルクの不足', 8.274), ('外的要因による損傷', 7.274), ('異物の混入', 6.27)]
パッキンの経年劣化
------------------------------------------------------
Item.Description              フィルター
Item.Manufacturer                D社
Item.PartsNo               P-998342
問題点.内容                      パッキンの劣化
問題点.発覚日時                 2025/07/05
問題点.Project          塩素注入設備更新プロジェクト
Name: 1, dtype: object
[('電源系統の不安定', 8.289), ('異物の混入', 6.252), ('塩素ガスの侵入', 6.226)]
外的要因による損傷
------------------------------------------------------
Item.Description                逆止弁
Item.Manufacturer                E社
Item.PartsNo               P-756904
問題点.内容                      塩素漏れが発生
問題点.発覚日時                 2025/07/26
問題点.Project          塩素注入設備更新プロジェクト
Name: 2, dtype: object
[('外的要因による損傷', 8.29), ('塩素ガスの侵